<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic -q
!pip install colab-env --quiet

import anthropic
import os
import colab_env
import json

In [2]:
api_key = os.environ["CLAUDE3_API_KEY"]
#model="claude-3-opus-20240229"
model="claude-3-5-sonnet-20240620"

client = anthropic.Anthropic(
    api_key=api_key,
)


message = client.messages.create(
    #model="claude-3-opus-20240229",
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content[0].text)

Hello! It's nice to meet you. How can I help you today?


In [ ]:
#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("ryleymcconkey/ml-turbulence-dataset")

#print("Path to dataset files:", path)

In [10]:
#!cp -pr /root/.cache/kagglehub/datasets/ryleymcconkey/ml-turbulence-dataset/versions/6/REF.csv /content/gdrive/MyDrive/datasets/turbulence/REF.csv

In [4]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
turnulence_df = pd.read_csv('/content/gdrive/MyDrive/datasets/turbulence/REF.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
turnulence_df

In [6]:
import os
from sklearn.linear_model import LogisticRegression

# Import the Claude 3 API library
import anthropic

# Get the API key from the environment variable
api_key = os.environ["CLAUDE3_API_KEY"]
model="claude-3-5-sonnet-20240620"

# Initialize the Claude 3 client
client = anthropic.Anthropic(
    api_key=api_key,
)
class FlightEnvironment:
    def __init__(self, origin, destination, weather):
        self.origin = origin
        self.destination = destination
        self.weather = weather  # Stores predefined weather data

    def get_possible_routes(self):
        """
        Retrieves possible flight routes between the origin and destination.

        In a real-world scenario, this would involve querying a database or API.
        For simplicity, we'll return a few hardcoded routes.

        Returns:
            list: A list of dictionaries representing flight routes.
                 Each dictionary contains flight details (flight_number, departure_time, arrival_time, price).
                 Returns an empty list if no routes are found.
        """
        routes = {
            ("Montreal", "Toronto"): [
                {
                    "flight_number": "AC123",
                    "departure_time": "8:00",
                    "arrival_time": "9:30",
                    "price": 200,
                },
                {
                    "flight_number": "WJ456",
                    "departure_time": "12:00",
                    "arrival_time": "13:30",
                    "price": 250,
                },
            ],
            ("Montreal", "Vancouver"): [
                {
                    "flight_number": "AC789",
                    "departure_time": "9:00",
                    "arrival_time": "14:00",
                    "price": 500,
                }
            ],
        }
        return routes.get((self.origin, self.destination), [])  # Return empty list if no routes found

    def get_weather_data(self, location):
        """
        Retrieves weather data for a given location.

        Simulates fetching weather data from a dictionary.
        In a real-world scenario, this would involve querying a weather API.

        Args:
            location (str): The location for which to retrieve weather data.

        Returns:
            dict: A dictionary containing weather information (weather, wind).
                 Defaults to sunny weather with 5 m/s wind speed if location is not found.
        """
        return self.weather.get(
            location, {"weather": [{"description": "Sunny"}], "wind": {"speed": 5}}
        )

import pandas as pd
from sklearn.linear_model import LogisticRegression
import anthropic
import os

# Get the API key from the environment variable
api_key = os.environ["CLAUDE3_API_KEY"]
model = "claude-3-5-sonnet-20240620"

# Initialize the Claude 3 client
client = anthropic.Anthropic(api_key=api_key)


class FlightAgent:
    def __init__(self, environment, turbulence_dataset_path):
        self.environment = environment
        self.turbulence_model = LogisticRegression()
        self.turbulence_dataset_path = turbulence_dataset_path
        self.train_turbulence_model()  # Train the model during initialization

    def train_turbulence_model(self):
        turbulence_df = pd.read_csv(self.turbulence_dataset_path)

        # Print the available columns to check if 'turbulence' exists
        #print(turbulence_df.columns)

        # Add 'Turbulence' column based on 'REF_k'
        # (You might need to adjust the threshold based on your data)
        turbulence_df['Turbulence'] = (turbulence_df['REF_k'] > turbulence_df['REF_k'].mean()).astype(int)
        target = turbulence_df['Turbulence']



        # Select relevant features for training
        features = turbulence_df[
            [
                "REF_U_1",
                "REF_U_2",
                "REF_U_3",
                "REF_tau_11",
                "REF_tau_12",
                "REF_tau_13",
                "REF_tau_22",
                "REF_tau_23",
                "REF_tau_33",
            ]
        ]


        target = turbulence_df['Turbulence']

        # Train the model
        self.turbulence_model.fit(features, target)
        print("Turbulence model trained successfully!")

    def analyze_weather(self):
        # Observe: Gather weather data for origin and destination
        origin_weather = self.environment.get_weather_data(self.environment.origin)
        destination_weather = self.environment.get_weather_data(
            self.environment.destination
        )

        # Orient: Analyze weather data to identify potential risks
        print(
            f"Weather in {self.environment.origin}: {origin_weather['weather'][0]['description']}"
        )
        print(
            f"Weather in {self.environment.destination}: {destination_weather['weather'][0]['description']}"
        )

        # Use Claude 3 to analyze textual weather reports and identify potential risks
        prompt = f"""Analyze the following weather reports for potential risks to flight operations:

        Origin ({self.environment.origin}): {origin_weather}
        Destination ({self.environment.destination}): {destination_weather}
        """
        # In the FlightAgent class, within the analyze_weather method
        response = client.messages.create(
            model=model,
            max_tokens=1024,  # Changed from max_tokens_to_sample to max_tokens
            messages=[{"role": "user", "content": prompt}],
        )
        print("Claude 3 weather analysis:", response.content[0].text)

        # Extract relevant features for turbulence prediction
        origin_wind_speed = origin_weather["wind"]["speed"]
        destination_wind_speed = destination_weather["wind"]["speed"]


    def assess_flight_options(self):
        # Observe: Gather available flight options
        possible_routes = self.environment.get_possible_routes()

        # Orient: Evaluate and display flight options
        print("Available flights:")
        for route in possible_routes:
            print(route)

    def choose_flight(self):
        # Observe:
        self.analyze_weather()
        # Orient:
        self.assess_flight_options()
        # Decide:
        possible_routes = self.environment.get_possible_routes()
        if not possible_routes:
            return "No flights found."

        # Use Claude 3 to assist in decision-making, considering multiple factors
        prompt = f"""Given the following information:
        - Origin: {self.environment.origin}
        - Destination: {self.environment.destination}
        - Available flights: {possible_routes}
        - Weather conditions: (obtained from analyze_weather)
        - Passenger preferences: (not included in this example)

        Recommend the optimal flight plan.
        """
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            messages=[{"role": "user", "content": prompt}],
        )
        # print("Claude 3 flight recommendation:", response.content.text)
        chosen_flight = response.content[0].text

        # Act: (In this example, "acting" is printing the chosen flight)
        return chosen_flight

# Initialize environment and agent with weather data
environment = FlightEnvironment(
    origin="Montreal",
    destination="Toronto",
    weather={"Montreal": {"weather": [{"description": "Cloudy"}], "wind": {"speed": 10}},
             "Toronto": {"weather": [{"description": "Rainy"}], "wind": {"speed": 15}}}
)


# Provide the path to the turbulence dataset when creating the FlightAgent instance
agent = FlightAgent(environment, '/content/gdrive/MyDrive/datasets/turbulence/REF.csv')


# Run the agent
chosen_flight = agent.choose_flight()
print("Chosen flight:")
print(chosen_flight)

Turbulence model trained successfully!
Weather in Montreal: Cloudy
Weather in Toronto: Rainy
Claude 3 weather analysis: Based on the provided weather reports for Montreal (origin) and Toronto (destination), let's analyze the potential risks to flight operations:

1. Origin (Montreal):
   - Weather: Cloudy
   - Wind speed: 10 (units not specified, assumed to be knots or mph)

   Potential risks:
   - Reduced visibility due to cloud cover, which may affect takeoff procedures
   - Possible turbulence, especially if there are variations in cloud layers

2. Destination (Toronto):
   - Weather: Rainy
   - Wind speed: 15 (units not specified, assumed to be knots or mph)

   Potential risks:
   - Reduced visibility due to rain, which may affect landing procedures
   - Wet runway, increasing the risk of hydroplaning during landing
   - Possible wind shear or gusty conditions associated with rain
   - Higher wind speed may affect approach and landing stability
   - Increased turbulence, especial